# Module 1 - Explore Graph with Cypher

This module has the following objective:
- Running Cypher queries to better understand the data we've loaded

Import our usual suspects (and some more...)

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result

# Setup

Load env variables

In [ ]:
env_file = 'ws.env'

In [ ]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')
else:
    print(f"File {env_file} not found.")

Setup connection to the database with the [Python Driver](https://neo4j.com/docs/python-manual/5/).

In [ ]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the connection

In [ ]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

## Explore the graph 

Now to the database and observe what is there. Example queries:

- `MATCH (n:Customer) RETURN n LIMIT 25`

- `MATCH (n:Chunk) RETURN n LIMIT 25`

- `MATCH p=()-[r:PART_OF]->() RETURN p LIMIT 25`

We can also run this via the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/5/) wihtin your Notebook. Let's do so below!

### How many Customers are there in the graph?

In [ ]:
driver.execute_query(
    """
    MATCH (cust:Customer) 
    RETURN COUNT(cust)
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

### What products do customers have? 

In [ ]:
customer_product_df = driver.execute_query(
    """
    MATCH (cust:Customer)-[:HAS_PRODUCT]->(prod:Product)
    WITH DISTINCT cust, COLLECT(prod.name) AS products
    RETURN cust.name AS customer_name, SIZE(products) AS num_products, products LIMIT 10
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
customer_product_df

### How many products on average?

In [ ]:
customer_product_df["num_products"].describe()

### What are the most common products?

In [ ]:
product_df = driver.execute_query(
    """
    MATCH (cust:Customer)-[:HAS_PRODUCT]->(prod:Product)
    RETURN DISTINCT prod.name AS product_name, COUNT(cust) AS num_customers ORDER BY num_customers DESC
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
product_df

### What documents are in the database?

In [ ]:
document_df = driver.execute_query(
    """
    MATCH (prod_type:ProductType)<-[:RELATED_TO]-(doc:Document) 
    RETURN DISTINCT prod_type.name AS product_type, doc.file_name AS document, doc.pages AS num_pages
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
document_df

### How many chunks are the documents split into?

In [ ]:
document_chunks_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)
    WITH DISTINCT doc, COLLECT(chunk) AS chunks
    RETURN doc.file_name AS file_name, doc.pages AS pages, SIZE(chunks) AS chunk_count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
document_chunks_df

## Extracted Definitions

Let's dive into the extracted definitions. 

In [ ]:
definitions_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)-[:MENTIONS]->(def:Definition)
    WITH DISTINCT doc, COLLECT(def) AS definitions
    RETURN doc.file_name AS file_name, SIZE(definitions) AS definition_count, [def in definitions | def.term][0] AS definition_example, [def in definitions | def.description][0] AS description_example
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [ ]:
definitions_df

### What definitions are mentioned most frequently within chunks?

Some definitions are mentioned in many chunks. These "supernodes" are often not informative as they do not capture specific information about the particular chunk itself.

In [ ]:
definition_count_df = driver.execute_query(
    """
    MATCH (doc:Document)<-[:PART_OF]-(chunk:Chunk)-[:MENTIONS]->(def:Definition)
    WITH DISTINCT def, COUNT(DISTINCT chunk) AS chunk_count, COUNT(DISTINCT doc) AS document_count
    RETURN def.term AS definition, def.description AS description, chunk_count, document_count ORDER BY chunk_count DESC LIMIT 25
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
)

In [ ]:
definition_count_df